## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key =g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Pangnirtung,66.1451,-65.7125,42.80,75,75,17.27,CA,broken clouds
1,1,Olinda,-8.0089,-34.8553,76.73,73,40,12.66,BR,scattered clouds
2,2,Payakumbuh,-0.2202,100.6308,68.54,95,100,1.70,ID,overcast clouds
3,3,Hermanus,-34.4187,19.2345,53.80,79,49,3.09,ZA,scattered clouds
4,4,Nizhneivkino,58.1940,49.5214,53.29,81,100,12.44,RU,overcast clouds


In [4]:
min_temp = float(input("What is the min temp?"          ))
                       
max_temp = float(input("What is the max temp?"          ))


What is the min temp?65
What is the max temp?90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_df= city_data_df.loc[(city_data_df['Max Temp']>=min_temp)&(city_data_df['Max Temp']<=max_temp)]

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# 4a. Determine if there are any empty rows.
filtered_df= filtered_df.dropna()

In [13]:
# 4a. Determine if there are any empty rows.
filtered_df.count()

City_ID                359
City                   359
Lat                    359
Lng                    359
Max Temp               359
Humidity               359
Cloudiness             359
Wind Speed             359
Country                359
Current Description    359
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Olinda,BR,76.73,scattered clouds,-8.0089,-34.8553,
2,Payakumbuh,ID,68.54,overcast clouds,-0.2202,100.6308,
6,Bambous Virieux,MU,70.21,light rain,-20.3428,57.7575,
7,Albany,US,65.64,light rain,42.6001,-73.9662,
8,Meulaboh,ID,75.27,moderate rain,4.1363,96.1285,
10,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
12,Nusaybin,TR,82.51,clear sky,37.0778,41.2178,
13,Butaritari,KI,82.49,overcast clouds,3.0707,172.7902,
14,Oistins,BB,82.49,broken clouds,13.0667,-59.5333,
20,Victoria,HK,85.32,scattered clouds,22.2855,114.1577,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
 # Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
   
         # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Olinda,BR,76.73,scattered clouds,-8.0089,-34.8553,Flat Quatro Rodas Olinda
2,Payakumbuh,ID,68.54,overcast clouds,-0.2202,100.6308,Dalimo Homestay
6,Bambous Virieux,MU,70.21,light rain,-20.3428,57.7575,Casa Tia Villa
7,Albany,US,65.64,light rain,42.6001,-73.9662,
8,Meulaboh,ID,75.27,moderate rain,4.1363,96.1285,Kost Pria


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [3]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [4]:
# 11a. Add a marker layer for each city to the map. 
##max_temp = hotel_df["Max Temp"]
#fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))